In [1]:
from bs4 import BeautifulSoup
from win32com import client as wc
import os
import docx

In [2]:
dir_doc = "C:/Users/Max/Documents/sud/doc/"
dir_info = "C:/Users/Max/Documents/sud/info/"
dir_case = "C:/Users/Max/Documents/sud/case/"

In [3]:
w = wc.Dispatch('Word.AppLication')
files = sorted(os.listdir(dir_doc))
for name in files:
    if name.endswith('.doc'):
        if not os.path.exists(dir_doc + name + 'x'):
            doc = w.Documents.Open(dir_doc + name)
            doc.SaveAs(dir_doc + name + 'x', 16)
            doc.Close()
w.Quit()    

In [4]:
def get_info(name):
    d = {}
    file = open(name, encoding='utf8')
    content = file.read()
    soup = BeautifulSoup(content, 'lxml')
    table = soup.body.find('div', {'class', 'cardsud_wrapper'})
    for row in table.childGenerator():
        if row.name!=None:        
            key = row.find_all('div')[0].text
            val = row.find_all('div')[1].text
            key = key.replace(' ','').replace('\n', '')
            val = val.replace('"', ' ').replace('«', ' ').replace('»', ' ').replace(',', ' ').replace('\n', '')
            val = val.strip()
            #print(key, val)
            d[key] = val
    return d

In [5]:
def getd(d, key):
    if key in d:
        return d[key]
    else:
        return ''

In [6]:
file = open('mosgorsud.csv', 'w', encoding='utf-8')
file.write('id,side,date1,date2,category,status,verdict,verdict_up,reason,text\n')

67

In [7]:
get_info(dir_case + '0000005.htm')

{'Уникальныйидентификатордела': '77RS0031-01-2020-009416-66',
 'Номердела': '33-30321/2022',
 'Стороны': 'Истец: Микеладзе И.Е.Ответчик: Бородин Д.В.',
 'Датапоступленияделавапелляционнуюинстанцию': '01.08.2022',
 'Датаокончания': '24.08.2022',
 'Номерсудебногосостава': '4',
 'Номерделавсуденижестоящейинстанции': '02-0180/2021',
 'Судпервойинстанции,судья': 'Хорошёвский районный суд (Лутохина Р.А.)',
 'Категориядела': '205 - Иски о взыскании сумм по договору займа  кредитному договору',
 'Результатрассмотрения': 'Возвращено  24.08.2022'}

In [8]:
#for n in range(1, 301):
#    name = str(n).zfill(7)
#    if os.path.exists('doc/' + name + '.docx'):
files = sorted(os.listdir(dir_doc))
for name in files:
    if name.endswith('.docx'):

        d = get_info(dir_info + name[0:8] + 'htm')
        file.write(getd(d, 'Уникальныйидентификатордела') + ',')
        file.write(getd(d, 'Стороны') + ',')
        file.write(getd(d, 'Датапоступления') + ',')
        file.write(getd(d, 'Датарассмотренияделавпервойинстанции') + ',')
        file.write(getd(d, 'Категориядела') + ',')
        file.write(getd(d, 'Текущеесостояние') + ',')
        file.write(getd(d, 'Решениеапелляции') + ',')        

        d = get_info(dir_case + name[0:8] + 'htm')
        file.write(getd(d, 'Результатрассмотрения') + ',')
        file.write(getd(d, 'Основаниерешениясуда') + ',')                
    
        text = ''
        doc = docx.Document(dir_doc + name)
        for p in doc.paragraphs:
            text += p.text + ' '
        text = text.replace('"', ' ').replace('«', ' ').replace('»', ' ').replace('\n', ' ')
        file.write('"' + text + '"\n')
file.close()